In [2]:
import subprocess
import pickle
import numpy as np
import os
import glob
import cv2
import shutil
import pandas as pd
import os

thumos14_cls_name = ['BK',
                     'BaseballPitch',
                     'BasketballDunk',
                     'Billiards',
                     'CleanAndJerk',
                     'CliffDiving',
                     'CricketBowling',
                     'CricketShot',
                     'Diving',
                     'FrisbeeCatch',
                     'GolfSwing',
                     'HammerThrow',
                     'HighJump',
                     'JavelinThrow',
                     'LongJump',
                     'PoleVault',
                     'Shotput',
                     'SoccerPenalty',
                     'TennisSwing',
                     'ThrowDiscus',
                     'VolleyballSpiking']


tvseries_cls_name = ['BK',
                     'Pick something up',
                     'Point',
                     'Drink',
                     'Stand up',
                     'Run',
                     'Sit down',
                     'Read',
                     'Smoke',
                     'Drive car',
                     'Open door',
                     'Give something',
                     'Use computer',
                     'Write',
                     'Close door',
                     'Go down stairway',
                     'Go up stairway',
                     'Throw something',
                     'Get in/out of car',
                     'Hang up phone',
                     'Eat',
                     'Answer phone',
                     'Clap',
                     'Dress up',
                     'Undress',
                     'Kiss',
                     'Fall/trip',
                     'Wave',
                     'Pour',
                     'Punch',
                     'Fire weapon']


In [3]:
def extract_frames(video_path, dset):
    #video_path : '/home/muser/tech/data_samples/thumos_vid/video_test_0000026'
    # img_path = '/home/muser/techQmark/result_vid/' + video_name + '/imgs'
    img_path = '/opt/ml/demo_video_making/demo_vid/extracted_frames/' + video_name + '/imgs'
    os.makedirs(img_path, exist_ok=True)
    if dset == 'thumos14':
        subprocess.call('ffmpeg -i {} -r 24/1 -qscale:v 2 {}/%06d.png'.format(video_path+'.mp4', img_path), shell=True)
    else:
        subprocess.call('ffmpeg -i {} -r 24/1 -qscale:v 2 {}/%06d.png'.format(video_path+'.mkv', img_path), shell=True)

In [4]:
def put_text(image_name, info, dset, resize=False):
    # info : [frame_num, prob, cls, label]
    image = cv2.imread(image_name)
    [height, width] = image.shape[:2]
    frame_num = info[0]
    totsecs = float(frame_num/24)
    hours, mins, secs = int(totsecs / 3600), int((totsecs % 3600) / 60), int((totsecs % 60))
    font_size = 1
    loc_w, loc_h = 0, 0
    if dset == 'thumos14':
        loc_w, loc_h = 5, 10
        font_size = 0.4
        font_thick = 1
        step = 20
    else:
        loc_w, loc_h = 5, 30
        font_size = 1.
        font_thick = 2
        step = 30
    cv2.putText(image, "{:02d}:{:02d}:{:02d}".format(hours, mins, secs), (loc_w, loc_h), cv2.FONT_HERSHEY_DUPLEX,
                    font_size, (255, 255, 255), font_thick)
    cv2.putText(image, "PRED: {} ({:.2f})".format(info[2], info[1]), (loc_w, loc_h+step), cv2.FONT_HERSHEY_DUPLEX, font_size, (255, 0, 0), font_thick)
    cv2.putText(image, "GT: {}".format(info[3]), (loc_w, loc_h + 2*step), cv2.FONT_HERSHEY_DUPLEX, font_size, (0, 0, 255), font_thick)
    cv2.imwrite(image_name, image)

In [5]:
def put_text_for_prof(image_name, info, dset, resize=False):
    # info : [frame_num, prob, cls, label]
    image = cv2.imread(image_name)
    [height, width] = image.shape[:2]
    frame_num = info[0]
    totsecs = float(frame_num/24)
    hours, mins, secs = int(totsecs / 3600), int((totsecs % 3600) / 60), int((totsecs % 60))
    font_size = 1
    loc_w, loc_h = 0, 0
    if dset == 'thumos14':
        loc_w, loc_h = 5, 10
        font_size = 0.4
        font_thick = 1
        step = 20
    else:
        loc_w, loc_h = 5, 30
        font_size = 1.
        font_thick = 2
        step = 30
    cv2.putText(image, info[3], (loc_w, loc_h), cv2.FONT_HERSHEY_DUPLEX, font_size, (0, 0, 255), font_thick)
    cv2.imwrite(image_name, image)

In [6]:
def put_text_for_aa(image_name, info, dset, resize=False):
    # info : [frame_num, prob, cls, label]
    image = cv2.imread(image_name)
    [height, width] = image.shape[:2]
    frame_num = info[0]
    totsecs = float(frame_num/24)
    hours, mins, secs = int(totsecs / 3600), int((totsecs % 3600) / 60), int((totsecs % 60))
    font_size = 1
    loc_w, loc_h = 0, 0
    if dset == 'thumos14':
        loc_w, loc_h = 5, 10
        font_size = 0.4
        font_thick = 1
        step = 20
    else:
        loc_w, loc_h = 5, 30
        font_size = 1.
        font_thick = 2
        step = 30
    cv2.putText(image, "{:02d}:{:02d}:{:02d}".format(hours, mins, secs), (loc_w, loc_h), cv2.FONT_HERSHEY_DUPLEX,
                    font_size, (255, 255, 255), font_thick)
    cv2.putText(image, "PRED: {} ({:.2f})".format(info[2], info[1]), (loc_w, loc_h+step), cv2.FONT_HERSHEY_DUPLEX, font_size, (255, 0, 0), font_thick)
    cv2.putText(image, "GT: {}".format(info[3]), (loc_w, loc_h + 2*step), cv2.FONT_HERSHEY_DUPLEX, font_size, (0, 0, 255), font_thick)
    cv2.putText(image, f"a1_PRED: {info[4]} / a1_GT: {info[5]}", (loc_w, loc_h + 3*step), cv2.FONT_HERSHEY_DUPLEX, font_size, (0, 0, 255), font_thick)
    cv2.putText(image, f"a2_PRED: {info[6]} / a2_GT: {info[7]}", (loc_w, loc_h + 4*step), cv2.FONT_HERSHEY_DUPLEX, font_size, (0, 0, 255), font_thick)
    cv2.imwrite(image_name, image)

In [7]:
def record_videos(result_path, video_name, action_cls, dset):
    frm_path = '/opt/ml/demo_video_making/demo_vid/frames_for_detection_demo/' + video_name + '/frms'
    img_path = '/opt/ml/demo_video_making/demo_vid/extracted_frames/' + video_name + '/imgs'

    # copy frames to put text in copied frames
    # subprocess.call('cp {}/* {}/'.format(img_path, frm_path), shell=True)

    nframes = len(glob.glob(frm_path + '/*.png'))
    all_videos_results = pd.read_csv(result_path)
    prop_idx = all_videos_results['vid name'] == video_name
    this_video_results = all_videos_results[prop_idx][['end_chunk', 'prob', 'pred_cls', 'GT_cls']].values
    for info in this_video_results:
        f_start = (int(info[0]) * 32) + 1
        f_end = (int(info[0])+1) * 32
        prob = float(info[1])
        cls = action_cls[int(info[2])]
        label = action_cls[int(info[3])]
        for i in range(f_start, f_end+1):
            image_name = frm_path + '/{:06d}.png'.format(i)
            if os.path.exists(image_name):
                put_text(image_name, [i, prob, cls, label], dset)
            else:
                print('### NO ###')

    # record a video from texted frames
    print('\n\n### Record a video from texted frames ###\n\n')
    subprocess.call(
        'ffmpeg -framerate 24 -i {}/%06d.png -c:v libx264 -profile:v high -level:v 4.0 -crf 18 -pix_fmt yuv420p {} '.
        format(frm_path, '/opt/ml/demo_video_making/demo_vid/result_demo_video/detection/' + video_name+'.mp4'), shell=True) # 24fps짜리 비디오로 만들기

In [66]:
def record_videos_for_prof(result_path, video_name, action_cls, dset):
    frm_path = '/opt/ml/demo_video_making/demo_vid/frames_for_detection_demo/' + video_name + '/frms'
    img_path = '/opt/ml/demo_video_making/demo_vid/extracted_frames/' + video_name + '/imgs'

    # copy frames to put text in copied frames
    # subprocess.call('cp {}/* {}/'.format(img_path, frm_path), shell=True)

    nframes = len(glob.glob(frm_path + '/*.png'))
    all_videos_results = pd.read_csv(result_path)
    prop_idx = all_videos_results['vid name'] == video_name
    this_video_results = all_videos_results[prop_idx][['end_chunk', 'prob', 'pred_cls', 'GT_cls']].values
    for info in this_video_results:
        f_start = (int(info[0]) * 32) + 1
        f_end = (int(info[0])+1) * 32
        prob = float(info[1])
        cls = action_cls[int(info[2])]
        label = action_cls[int(info[3])]
        for i in range(f_start, f_end+1):
            image_name = frm_path + '/{:06d}.png'.format(i)
            if os.path.exists(image_name):
                if label == 'BK':
                    continue
                put_text_for_prof(image_name, [i, prob, cls, label], dset)
            else:
                print('### NO ###')

    # record a video from texted frames
    print('\n\n### Record a video from texted frames ###\n\n')
    subprocess.call(
        'ffmpeg -framerate 24 -i {}/%06d.png -c:v libx264 -profile:v high -level:v 4.0 -crf 18 -pix_fmt yuv420p {} '.
        format(frm_path, '/opt/ml/demo_video_making/demo_vid/result_demo_video/detection/' + video_name+'.mp4'), shell=True) # 24fps짜리 비디오로 만들기

In [70]:
def record_videos_of_a1(result_path, video_name, action_cls, dset):
    frm_path = '/opt/ml/demo_video_making/demo_vid/frames_for_test_demo/' + video_name + '/frms'
    img_path = '/opt/ml/demo_video_making/demo_vid/extracted_frames/' + video_name + '/imgs'

    # copy frames to put text in copied frames
    # subprocess.call('cp {}/* {}/'.format(img_path, frm_path), shell=True)

    nframes = len(glob.glob(frm_path + '/*.png'))
    all_videos_results = pd.read_csv(result_path)
    prop_idx = all_videos_results['vid name'] == video_name
    this_video_results = all_videos_results[prop_idx][['end_chunk', 'prob', 'pred_cls', 'GT_cls', 'a1_pred_cls', 'a1_GT_cls', 'a2_pred_cls', 'a2_GT_cls']].values
    for info in this_video_results:
        f_start = (int(info[0]) * 32) + 1
        f_end = (int(info[0])+1) * 32
        prob = float(info[1])
        cls = action_cls[int(info[2])]
        label = action_cls[int(info[3])]
        a1_cls = action_cls[int(info[4])]
        a1_label = action_cls[int(info[5])]
        a2_cls = action_cls[int(info[6])]
        a2_label = action_cls[int(info[7])]
        for i in range(f_start, f_end+1):
            image_name = frm_path + '/{:06d}.png'.format(i)
            if os.path.exists(image_name):
                put_text_for_aa(image_name, [i, prob, cls, label, a1_cls, a1_label, a2_cls, a2_label], dset)
            else:
                print('### NO ###')

    # record a video from texted frames
    print('\n\n### Record a video from texted frames ###\n\n')
    os.makedirs('/opt/ml/demo_video_making/demo_vid/result_demo_video/detection/', exist_ok=True)
    subprocess.call(
        'ffmpeg -framerate 24 -i {}/%06d.png -c:v libx264 -profile:v high -level:v 4.0 -crf 18 -pix_fmt yuv420p {} '.
        format(frm_path, '/opt/ml/demo_video_making/demo_vid/result_demo_video/detection/' + video_name+'.mp4'), shell=True) # 24fps짜리 비디오로 만들기

In [13]:
def record_videos_of_a2(result_path, video_name, action_cls, dset):
    print(video_name)
    frm_path = '/opt/ml/demo_video_making/demo_vid/frames_for_detection_demo/' + video_name + '/frms'
    img_path = '/opt/ml/demo_video_making/demo_vid/extracted_frames/' + video_name + '/imgs'

    # copy frames to put text in copied frames
    # subprocess.call('cp {}/* {}/'.format(img_path, frm_path), shell=True)

    nframes = len(glob.glob(frm_path + '/*.png'))
    all_videos_results = pd.read_csv(result_path)
    prop_idx = all_videos_results['vid name'] == video_name
    this_video_results = all_videos_results[prop_idx][['end_chunk', 'prob', 'pred_cls', 'GT_cls']].values
    for info in this_video_results:
        f_start = (int(info[0]) * 32) + 1
        f_end = (int(info[0])+1) * 32
        prob = float(info[1])
        cls = action_cls[int(info[2])]
        label = action_cls[int(info[3])]
        for i in range(f_start, f_end+1):
            image_name = frm_path + '/{:06d}.png'.format(i)
            if os.path.exists(image_name):
                # print('------------------')
                # print('### PUT TEXT ####')
                # print('------------------')
                put_text(image_name, [i, prob, cls, label], dset)
            else:
                print('### NO ###')

    # record a video from texted frames
    print('\n\n### Record a video from texted frames ###\n\n')
    os.makedirs('/opt/ml/demo_video_making/demo_vid/result_demo_video/detection/', exist_ok=True)
    subprocess.call(
        'ffmpeg -framerate 24 -i {}/%06d.png -c:v libx264 -profile:v high -level:v 4.0 -crf 18 -pix_fmt yuv420p {} '.
        format(frm_path, '/opt/ml/demo_video_making/demo_vid/result_demo_video/detection/' + video_name+'.mp4'), shell=True) # 24fps짜리 비디오로 만들기

In [8]:
dataset = 'thumos14'
alpha = 0.75
vid_type='video'

if dataset == 'thumos14':
    video_dir = '/opt/ml/demo_video_making/data_samples/thumos/'
    if alpha == 0.75:
        result_path = '/opt/ml/demo_video_making/demo_vid/result_csv/test_alpha0.75_tsm_thumos14.csv'
    else:
        result_path = '/opt/ml/demo_video_making/demo_vid/result_csv/test_alpha1.0_tsm_thumos14.csv'
    video_lst = ['video_test_0000179.mp4', 'video_test_0000131.mp4', 'video_test_0000964.mp4', 'video_test_0000372.mp4',
                'video_test_0000443.mp4', 'video_test_0000882.mp4', 'video_test_0000950.mp4', 'video_test_0000964.mp4',
                'video_test_0001058.mp4', 'video_test_0001072.mp4'] # 
    print('\nMake demo video about {}\n'.format(video_lst[0]))
    act_cls = thumos14_cls_name
else:
    video_dir = '/opt/ml/demo_video_making/data_samples/tvseries/'
    if alpha == 0.75:
        result_path = '/opt/ml/demo_video_making/demo_vid/result_csv/test_alpha0.75_tsm_tvseries.csv'
    else:
        result_path = '/opt/ml/demo_video_making/demo_vid/result_csv/test_alpha1.0_tsm_tvseries.csv'
    video_lst = ['How_I_Met_Your_Mother_ep8.mkv']
    act_cls = tvseries_cls_name

# for idx, video_name in enumerate(video_lst):
video_name = video_lst[0]
video_name = video_name[:-4] # 확장자 뺀 이름값만 추출
if os.path.isdir('result_demo_video/' + video_name): # result_vid 안에 비디오 이름으로 된 디렉터리 있으면
    shutil.rmtree('result_demo_video/' + video_name, ignore_errors=True) # 싹다제거
print(f'### VIDEO NAME ### : {video_name}')
os.makedirs(os.path.join('/opt/ml/demo_video_making/demo_vid/frames_for_detection_demo',video_name,'frms'), exist_ok=True)
print('\n### extract frames from the video... ###\n')
extract_frames(video_dir + video_name, dataset) # video 이름으로 된 폴더 만들고 그안에서 프레임 추출


Make demo video about video_test_0000179.mp4

### VIDEO NAME ### : video_test_0000179

### extract frames from the video... ###



ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.5.0 (crosstool-NG 1.24.0.131_87df0e6_dirty)
  configuration: --prefix=/opt/conda --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1602879523915/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/opt/ml/demo_video_making/data_s

In [ ]:
record_videos()

In [72]:
# a1, a2 성능 테스트 해볼 비디오 만드는 셀
dataset = 'tvseries'
alpha = 0.75
vid_type='video'

if dataset == 'thumos14':
    video_dir = '/opt/ml/demo_video_making/data_samples/thumos/'
    if alpha == 0.75:
        result_path = '/opt/ml/demo_video_making/demo_vid/result_csv/test_alpha0.75_tsm_thumos14.csv'
    else:
        result_path = '/opt/ml/demo_video_making/demo_vid/result_csv/test_alpha1.0_tsm_thumos14.csv'
    video_lst = ['video_test_0000250.mp4','video_test_0000026.mp4','video_test_0000045.mp4','video_test_0000129.mp4'] # 
    print('\nMake demo video about {}\n'.format(video_lst[0]))
    act_cls = thumos14_cls_name
else:
    video_dir = '/opt/ml/demo_video_making/data_samples/tvseries/'
    if alpha == 0.75:
        result_path = '/opt/ml/demo_video_making/demo_vid/result_csv/test_alpha0.75_tsm_tvseries.csv'
    else:
        result_path = '/opt/ml/demo_video_making/demo_vid/result_csv/test_alpha1.0_tsm_tvseries.csv'
    video_lst = ['How_I_Met_Your_Mother_ep8.mkv', 'Mad_Men_ep3.mkv']
    act_cls = tvseries_cls_name

# for idx, video_name in enumerate(video_lst):
video_name = video_lst[0]
video_name = video_name[:-4] # 확장자 뺀 이름값만 추출
if os.path.isdir('result_demo_video/' + video_name): # result_vid 안에 비디오 이름으로 된 디렉터리 있으면
    shutil.rmtree('result_demo_video/' + video_name, ignore_errors=True) # 싹다제거
print(f'### VIDEO NAME ### : {video_name}')
os.makedirs(os.path.join('/opt/ml/demo_video_making/demo_vid/frames_for_test_demo',video_name,'frms'), exist_ok=True)
print('\n### extract frames from the video... ###\n')
extract_frames(video_dir + video_name, dataset) # video 이름으로 된 폴더 만들고 그안에서 프레임 추출

### VIDEO NAME ### : How_I_Met_Your_Mother_ep8

### extract frames from the video... ###



In [68]:
record_videos_for_prof(result_path, video_name, act_cls, dataset)



### Record a video from texted frames ###




ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.5.0 (crosstool-NG 1.24.0.131_87df0e6_dirty)
  configuration: --prefix=/opt/conda --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1602879523915/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Input #0, image2, from '/opt/ml/demo_video_making/demo_vid/frames_for_det

In [73]:
record_videos_of_a1(result_path, video_name, act_cls, dataset)



### Record a video from texted frames ###




ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.5.0 (crosstool-NG 1.24.0.131_87df0e6_dirty)
  configuration: --prefix=/opt/conda --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1602879523915/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Input #0, image2, from '/opt/ml/demo_video_making/demo_vid/frames_for_tes

In [48]:
from scipy import io
mat_file = io.loadmat('/opt/ml/demo_video_making/data_samples/test_set_meta.mat')
print(mat_file['test_videos'][0][1][2])
vid_list = []
for i in mat_file['test_videos'][0]:
    if i[2][0] == 'HammerThrow':
        vid_list.append(i[0][0])
print(vid_list)
    

['HorseRiding']
['video_test_0000371', 'video_test_0000372', 'video_test_0000443', 'video_test_0000608', 'video_test_0000654', 'video_test_0000842', 'video_test_0000882', 'video_test_0000950', 'video_test_0001058', 'video_test_0001072', 'video_test_0001137', 'video_test_0001164', 'video_test_0001184', 'video_test_0001229', 'video_test_0001282', 'video_test_0001431']


In [5]:
# a1, a2 성능 테스트 해볼 비디오 만드는 셀
dataset = 'tvseries'
alpha = 0.75
vid_type = 'video'

if dataset == 'thumos14':
    video_dir = '/opt/ml/demo_video_making/data_samples/thumos/'
    if alpha == 0.75:
        result_path = '/opt/ml/demo_video_making/demo_vid/result_csv/test_alpha0.75_tsm_thumos14.csv'
    else:
        result_path = '/opt/ml/demo_video_making/demo_vid/result_csv/test_alpha1.0_tsm_thumos14.csv'
    video_lst = ['video_test_0000250.mp4','video_test_0000026.mp4','video_test_0000045.mp4','video_test_0000129.mp4'] # 
    print('\nMake demo video about {}\n'.format(video_lst[0]))
    act_cls = thumos14_cls_name
else:
    video_dir = '/opt/ml/demo_video_making/data_samples/tvseries/'
    if alpha == 0.75:
        result_path = '/opt/ml/demo_video_making/demo_vid/result_csv/test_alpha0.75_tsm_tvseries.csv'
    else:
        result_path = '/opt/ml/demo_video_making/demo_vid/result_csv/test_alpha1.0_tsm_tvseries.csv'
    video_lst = ['24_ep4.mkv', 'Breaking_Bad_ep3.mkv', 'Mad_Men_ep3.mkv', 'Sons_of_Anarchy_ep3.mkv']
    act_cls = tvseries_cls_name

# for idx, video_name in enumerate(video_lst):
video_name = video_lst[3]
video_name = video_name[:-4] # 확장자 뺀 이름값만 추출
if os.path.isdir('result_demo_video/' + video_name): # result_vid 안에 비디오 이름으로 된 디렉터리 있으면
    shutil.rmtree('result_demo_video/' + video_name, ignore_errors=True) # 싹다제거
print(f'### VIDEO NAME ### : {video_name}')
os.makedirs(os.path.join('/opt/ml/demo_video_making/demo_vid/frames_for_test_demo', video_name,'frms'), exist_ok=True)
print('\n### extract frames from the video... ###\n')
# extract_frames(video_dir + video_name, dataset) # video 이름으로 된 폴더 만들고 그안에서 프레임 추출

### VIDEO NAME ### : Sons_of_Anarchy_ep3

### extract frames from the video... ###



In [9]:
file_list = sorted(os.listdir('/opt/ml/demo_video_making/demo_vid/frames_for_test_demo/Mad_Men_ep3/frms'))
for i in range(len(file_list)):
    if int(file_list[i].split('.')[0]) != i+1:
        print(file_list[i])
        break